# **SHARAD REST API Downloader**
Author: g.nodjoumi@jacobs-university.de 

## What is it?
This notebook can be used to download search and download single to multiple SHARAD acquisition (IMG+lbl file) from ODE-PDS, including US Surface Clutter simulations.

## How-TO

* Customize [User input](#User-input) by defining:
    * destination folder
    * US SCS download (default True)
    * bounding box (left, bottom, right, top)
    * data type (EDR, RDRv2, USRDRv2)
    * Edit WCS_url and bmap_layerid if custom WCS is available
* Use the interactive box select tool, to select specific tracks
* Continue execution until [Start Download](#Start-download) to start the download    

## Limitations

* Although the notebook works for polar regions (<-85° and > 85°), the visualization has some issues.

## To-Do

* [ ] Implement Polar visualization
____________________________________
**Funding**: This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.

**Derived from GMAP-GPT Examples [1](https://github.com/europlanet-gmap/exemplary-notebooks/blob/main/pds/01_search-download_from_ode-pds/01_search_data_products.ipynb) and [2](https://github.com/europlanet-gmap/exemplary-notebooks/blob/main/pds/01_search-download_from_ode-pds/01_search_data_products.ipynb)**

In [ ]:
import gpt #Clone and install https://github.com/chbrandt/gpt
from gpt.search import ode
import geopandas as gpd
import geoviews as gv
gv.extension('bokeh', 'matplotlib')
import holoviews as hv
from holoviews.streams import BoundsXY
import os
import rioxarray as riox
from shapely.geometry import box
from tqdm import tqdm
from utils.download_utils import chunk_creator, download, download_checker, user_filter,downloader_basemap_prep, get_products
from utils.utils import get_poly
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning) # Only temporary to suppress cartopy and shapely futurewarnings
def selects(bounds):
    bbox = box(bounds[0],bounds[1],bounds[2],bounds[3])
    filtered_names = gpd.clip(track_gdf,mask=bbox)['FileName']
    filtered=track_gdf[track_gdf['FileName'].isin(filtered_names)]
    return (gv.Path(filtered,vdims=['pdsid']).opts(height=plot_height,show_grid=True, alpha=1, color = '#7cfc00'))

## Available data

In [ ]:
SHARADsets = ode.available_datasets('mars')
SHARADsets.loc['MRO'].loc['SHARAD']

## User input

In this section, user must select the destination folder ***ddir***, the data type ***Data***, the bounding box coordinates in -180+180 reference system (***min_Lon***, ***min_Lat***, ***max_Lat***, ***max_Lon***), and, if necessary change the ***wcs_url*** and ***bmap_layerid***.

In [ ]:
ddir = './Data/bubba' # Name of the folder where to download files
os.makedirs(ddir, exist_ok=True)
download_simulations = True # Download SHARAD US Surface Clutter Simulation 
#bounding_box=[0, 80, 10, 90] # Bounding box for searching data (Left, Bottom, Right, Top)
Data = 'EDR' # EDR, USRDRv2, RDR
min_Lon = -10
min_Lat = -10
max_Lat = 10
max_Lon = 10
######## edit if you want to use custom WCS service
wcs_url ='https://explore.jacobs-university.de/geoserver/ows?service=WCS' # Temporary default WCS service. Is it possible to replace with custom one.
bmap_layerid = 'Mars_Viking_MDIM21_ClrMosaic_global_232m_crs_cog' # IF wcs_url is not the default one, change the layer name accordingly 

######## edit only if plot visualization problems occurs
plot_height = 720

## Track footprints

In following section:
* ***get_products***: use gpt python package combined with the bounding box and data type to access PDS data archive and retreive a geopackage containing all available data information. If the *download_simulations* variable is True, a list of SHARAD USGS SCS simulations will be retrieved.
* ***downloader_basemap_prep***: use the footprint of the geopackage to download the basemap from WCS service. Then creates the geoviews plots.
* ***get_poly***: create a plot-compatible polygon of the user bounding box.
* ***rgb.opts***: plot the basemap, the user bounding box, and all available tracks.

In [ ]:
def parallel_df(files, ddir, jobs):
    from joblib import Parallel, delayed
    results = Parallel (n_jobs=jobs)(delayed(download)(files[i], ddir)
                            for i in range(len(files)))    

In [ ]:
file_list, track_gdf = get_products([min_Lon, min_Lat, max_Lon, max_Lat], 'MRO', 'SHARAD', Data, download_simulations)
track_gdf = track_gdf.drop_duplicates('geometry')
bounding_box = [min_Lon, min_Lat, max_Lon, max_Lat]
limits = [el*100000 for el in bounding_box]
try:
    bmap_name, ximg, plot_width, rgb,map_error = downloader_basemap_prep(wcs_url, bmap_layerid, ddir, min_Lon, min_Lat, max_Lon, max_Lat, plot_height, resx=0.05,resy=0.05)
except Exception as map_error:
    bmap_name=None
    print(f'Basemap not available or not acquired: {map_error}')
poly = get_poly([min_Lon, min_Lat, max_Lon, max_Lat])
#polygons = gv.Path([gv.Shape(geom) for geom in track_gdf.geometry]).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
polygons = gv.Path(data=track_gdf,vdims=['pdsid']).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red',tools=["hover", "lasso_select", "box_select"])
#polygons = gv.Path(track_gdf.geometry).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
stream = BoundsXY(source=polygons)
sel = hv.streams.BoundsXY(source=polygons,bounds=(min_Lon,min_Lat,max_Lon,max_Lat))
dmap = hv.DynamicMap(selects, streams=[sel])
polygons*dmap

if bmap_name == None:
    layout = gv.Polygons(poly).opts(alpha=0.2,height=plot_height, width = plot_height*2)*polygons*dmap
else:
    
    layout = rgb.opts(height=plot_height, width = plot_height*2,show_grid=True).opts(toolbar='left')*gv.Polygons(poly).opts(alpha=0.2)*polygons*dmap
    
layout.opts(xlim=(limits[0],limits[2]),ylim=(limits[1],limits[3]))

**IF NEEDED USE BOX SELECT TOOL IN THE ABOVE PLOT TO SELECT SPECIFIC TRACKS OTHERWISE SKIP TO DOWNLOAD**

In [ ]:
#Hang the execution to let the user draw subsurface layers, if present.
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

## **Download**

In [ ]:
track_names=[pdsid.lower().split('_rgram')[0] for pdsid in dmap[:,:].data['pdsid']]
filtered_list=user_filter(track_names,file_list)
dlist = download_checker(filtered_list,ddir)
chunks, jobs = chunk_creator(dlist)
with tqdm(total=len(dlist),
         desc = 'Generating files',
         unit='File') as pbar:
    for i in range(len(chunks)):
        files = chunks[i]    
        # print(files)
        parallel_df(files,ddir, jobs)        
        pbar.update(jobs)